In [346]:
# Census 1 
'''
Select an individual feature by attribute value from one feature class, then select features from another feature class by distance from the original feature. 
Originally used for selecting census block groups within a specified distance of a point feature. 
'''
def select_feature_by_attribute(focus_features, focus_field, focus_value): 
    
    # Construct SQL 
    sql = f"{focus_field} = '{focus_value}'"

    # Apply attribute selection 
    focus_point, count          = arcpy.management.SelectLayerByAttribute(focus_features, 'NEW_SELECTION', sql)
    if count == '0': 
        print('No focus features selected')
    elif count != 1: 
        print(f'Feature selected: {focus_value}')
        pass
    else: 
        print(f'Multiple features selected: {count}')
        
    return focus_point

In [347]:
def export_polygons_by_distance(polygon_input, selection_distance, polygon_output_folder, polygon_output_name): 
    
    # Select by distance 
    polygon_selection, merge, count = arcpy.management.SelectLayerByLocation(polygon_input, 'WITHIN_A_DISTANCE', focus_point, selection_distance)
    print(f'Selected  : {count} features')
    
    # Designate output path 
    polygon_output_path = os.path.join(polygon_output_folder, polygon_output_name)
    
    # Export target selection 
    try: 
        arcpy.conversion.ExportFeatures(polygon_selection, polygon_output_path)
    except: 
        print('boop')
        arcpy.management.Delete(polygon_output_path)
        arcpy.conversion.ExportFeatures(polygon_selection, polygon_output_path)
        
    print(f'Exported  : {polygon_output_path}')

    return polygon_output_path 

In [348]:
def create_selection_distance_dictionary(starting_hi, ending_hi, interval): 
    
    selection_distance_dictionary = {} 

    for i in range(starting_hi, ending_hi, interval): 
        selection_distance_dictionary[i] = i + interval
        
    return selection_distance_dictionary

In [355]:
def export_from_selection_distance_dictionary(selection_distance_dictionary, polygon_input, focus_point, export_folder, export_name, map_name): 
    
    # Initialize list of exports 
    export_paths = []
    
    # Iterate through dictionary 
    for low_distance, high_distance in selection_distance_dictionary.items(): 

        # High selection distance 
        selection_distance = f"{high_distance} Miles"
        print(selection_distance)

        # Make initial selection 
        high_selection, merge, high_count = arcpy.management.SelectLayerByLocation(polygon_input, 'WITHIN_A_DISTANCE', focus_point, selection_distance)

        print(f'{high_count} features selected at {selection_distance}')

        if low_distance != 0: 
            # Low selection distance 
            selection_distance = f"{low_distance} Miles"

            # Remove selection 
            low_selection, merge, low_count  = arcpy.management.SelectLayerByLocation(high_selection, 'WITHIN_A_DISTANCE', focus_point, selection_distance, selection_type = 'REMOVE_FROM_SELECTION')
            print(f'{low_count} features removed from selection')

            # Export 
            export_path   = os.path.join(export_folder, f'{export_name}_{high_distance}.shp')
            arcpy.conversion.ExportFeatures(low_selection, export_path)

        else: 
            low_count = 0 

            # Export 
            export_path   = os.path.join(export_folder, f'Minneapolis_{high_distance}.shp')
            arcpy.conversion.ExportFeatures(high_selection, export_path)

        # Map 
        mp = aprx.listMaps(map_name)[0]
        mp.addDataFromPath(export_path)
        
        # Exports 
        export_paths.append(export_path)
            
    return export_paths 

In [356]:
'''
Parameters 
'''
# Find Focus Point 
focus_features  = r'C:\DALE\Personal\Minnesota\MyProject\ACS.gdb\City_Points'
focus_field     = 'NAME'
focus_value     = 'Chicago'

# Export Block Groups 
export_name           = ""
polygon_input         = r'C:\DALE\Personal\Minnesota\GIS Projects\ACS\Block_Groups.gdb\ACS'
polygon_output_folder = r'C:\DALE\Personal\Minnesota\GIS Projects\ACS\Output Data\Block Group Shapefiles'
polygon_output_name   = focus_value + " " + selection_distance + '.shp'

# Map 
map_name        = 'Census'

# Selection Distance dictionary 
low       = 0
high      = 50
interval  = 5

In [357]:
'''
Export block groups by window 
'''

# Select focus feature 
focus_point = select_feature_by_attribute(focus_features, focus_field, focus_value)

# Create export folder 
export_folder = os.path.join(polygon_output_folder, focus_value)

try: 
    os.mkdir(export_folder)
ex
    
# Create distance dictionary
selection_distance_dictionary = create_selection_distance_dictionary(0, 50, 5)

Feature selected: Chicago


In [360]:
os.mkdir(os.path.join(polygon_output_folder, focus_value))

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\DALE\\Personal\\Minnesota\\GIS Projects\\ACS\\Output Data\\Block Group Shapefiles\\Chicago'

In [358]:
export_paths                  = export_from_selection_distance_dictionary(selection_distance_dictionary, polygon_input, focus_point, export_folder, focus_value, map_name)

5 Miles
468 features selected at 5 Miles


TypeError: expected str, bytes or os.PathLike object, not NoneType